In [82]:
import os
import requests
import inflection
import pandas as pd
import numpy as np
from pathlib import Path  

# 1.0 HEADERS

In [20]:
url = 'https://www.fundsexplorer.com.br/ranking'
headers = {    
    'User-Agent': 
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'
        ' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'    
}

In [21]:
response = requests.get(url, headers=headers)
if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]

In [22]:
df.sort_values('Códigodo fundo', inplace=True)
df.head()

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,...,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
102,ABCP11,Shoppings,"R$ 73,61",830.0,"R$ 0,55","0,72%","2,28%","4,65%","9,31%","0,76%",...,"R$ 1.119.952.411,71","R$ 91,77",80.0,"0,60%","0,02%","0,62%","5,14%","7,50%",NaN,1
286,AFHI11,Títulos e Val. Mob.,"R$ 97,00",6653.0,"R$ 1,15","1,19%","3,90%","7,93%","15,36%","1,30%",...,"R$ 287.450.160,21","R$ 95,20",102.0,"1,21%","-2,09%","-0,91%","2,26%",NaN,NaN,0
284,AFOF11,Títulos e Val. Mob.,"R$ 91,60",768.0,"R$ 1,08","1,18%","3,58%","7,11%","13,36%","1,19%",...,"R$ 69.465.153,20","R$ 100,50",91.0,"1,07%","2,33%","3,43%","8,23%",NaN,NaN,0
193,AIEC11,Lajes Corporativas,"R$ 79,07",9534.0,"R$ 0,73","0,91%","2,94%","6,30%","11,25%","0,98%",...,"R$ 489.178.047,14","R$ 101,38",78.0,"0,72%","0,04%","0,76%","5,83%","0,00%",NaN,2
3,ALMI11,Lajes Corporativas,"R$ 910,00",18.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 250.158.727,48","R$ 2.250,09",40.0,NaN,NaN,NaN,NaN,"64,05%",NaN,1


# 1.1 Rename Columns

In [23]:
df.columns

Index(['Códigodo fundo', 'Setor', 'Preço Atual', 'Liquidez Diária',
       'Dividendo', 'DividendYield', 'DY (3M)Acumulado', 'DY (6M)Acumulado',
       'DY (12M)Acumulado', 'DY (3M)Média', 'DY (6M)Média', 'DY (12M)Média',
       'DY Ano', 'Variação Preço', 'Rentab.Período', 'Rentab.Acumulada',
       'PatrimônioLíq.', 'VPA', 'P/VPA', 'DYPatrimonial',
       'VariaçãoPatrimonial', 'Rentab. Patr.no Período',
       'Rentab. Patr.Acumulada', 'VacânciaFísica', 'VacânciaFinanceira',
       'QuantidadeAtivos'],
      dtype='object')

In [24]:

old_columns = ['CodigoFundo', 'Setor', 'PrecoAtual', 'LiquidezDiaria',
       'Dividendo', 'DividendYield', 'DY3MAcumulado', 'DY6MAcumulado',
       'DY12MAcumulado', 'DY3MMedia', 'DY6MMedia', 'DY12MMedia',
       'DYAno', 'VariacaoPreço', 'RentabPeriodo', 'RentabAcumulada',
       'PatrimonioLíq', 'VPA', 'P_VPA', 'DYPatrimonial',
       'VariaçãoPatrimonial', 'RentabPatrPeriodo',
       'RentabPatrAcumulada', 'VacanciaFisica', 'VacanciaFinanceira',
       'QuantidadeAtivos']
snakecase = lambda x: inflection.underscore( x )

new_columns = list( map( snakecase, old_columns ) )

# rename
df.columns = new_columns
df.columns

Index(['codigo_fundo', 'setor', 'preco_atual', 'liquidez_diaria', 'dividendo',
       'dividend_yield', 'dy3_m_acumulado', 'dy6_m_acumulado',
       'dy12_m_acumulado', 'dy3_m_media', 'dy6_m_media', 'dy12_m_media',
       'dy_ano', 'variacao_preço', 'rentab_periodo', 'rentab_acumulada',
       'patrimonio_líq', 'vpa', 'p_vpa', 'dy_patrimonial',
       'variação_patrimonial', 'rentab_patr_periodo', 'rentab_patr_acumulada',
       'vacancia_fisica', 'vacancia_financeira', 'quantidade_ativos'],
      dtype='object')

# 2.0 DATA ENGINEERING

In [40]:
df1 = df.copy()

In [41]:
df1.isna().sum()

codigo_fundo               0
setor                      5
preco_atual                7
liquidez_diaria            7
dividendo                  0
dividend_yield             6
dy3_m_acumulado            6
dy6_m_acumulado            6
dy12_m_acumulado           6
dy3_m_media                6
dy6_m_media                6
dy12_m_media               6
dy_ano                    26
variacao_preço             6
rentab_periodo             6
rentab_acumulada           6
patrimonio_líq             0
vpa                        0
p_vpa                      7
dy_patrimonial            41
variação_patrimonial      41
rentab_patr_periodo       41
rentab_patr_acumulada     41
vacancia_fisica          168
vacancia_financeira      276
quantidade_ativos          0
dtype: int64

## 2.0.1 Remove NA

In [42]:
df1 = df1.dropna( axis=0, subset=['setor']  )

In [43]:
df1['setor'].unique()

array(['Shoppings', 'Títulos e Val. Mob.', 'Lajes Corporativas',
       'Logística', 'Híbrido', 'Outros', 'Hospital', 'Residencial',
       'Hotel'], dtype=object)

In [44]:
categorical_columns = ['codigo_fundo','setor']

In [46]:
df1[categorical_columns] = df1[categorical_columns].astype('category')

In [47]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 102 to 37
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   codigo_fundo           301 non-null    category
 1   setor                  301 non-null    category
 2   preco_atual            294 non-null    object  
 3   liquidez_diaria        294 non-null    float64 
 4   dividendo              301 non-null    object  
 5   dividend_yield         295 non-null    object  
 6   dy3_m_acumulado        295 non-null    object  
 7   dy6_m_acumulado        295 non-null    object  
 8   dy12_m_acumulado       295 non-null    object  
 9   dy3_m_media            295 non-null    object  
 10  dy6_m_media            295 non-null    object  
 11  dy12_m_media           295 non-null    object  
 12  dy_ano                 275 non-null    object  
 13  variacao_preço         295 non-null    object  
 14  rentab_periodo         295 non-null    ob

In [48]:
df1.isna().sum()

codigo_fundo               0
setor                      0
preco_atual                7
liquidez_diaria            7
dividendo                  0
dividend_yield             6
dy3_m_acumulado            6
dy6_m_acumulado            6
dy12_m_acumulado           6
dy3_m_media                6
dy6_m_media                6
dy12_m_media               6
dy_ano                    26
variacao_preço             6
rentab_periodo             6
rentab_acumulada           6
patrimonio_líq             0
vpa                        0
p_vpa                      7
dy_patrimonial            41
variação_patrimonial      41
rentab_patr_periodo       41
rentab_patr_acumulada     41
vacancia_fisica          163
vacancia_financeira      271
quantidade_ativos          0
dtype: int64

In [61]:
# pega todas as colunas que serão convertidas para o tipo float
#col_floats = df1.select_dtypes( exclude=['category', 'int64'] )
col_floats = list(df1.iloc[:,2:-1].columns)

# preenche os nans com 0
df1[col_floats] = df1[col_floats].fillna(value=0)

# normaliza os dados deixando apenas números
df1[col_floats] = df1[col_floats].applymap(lambda x: str(x).replace('R$', '').replace('.0','').replace('.','').replace('%','').replace(',','.'))

# altera o tipo para float
df1[col_floats] = df1[col_floats].astype('float')

In [66]:
df1.isna().sum()

codigo_fundo             0
setor                    0
preco_atual              0
liquidez_diaria          0
dividendo                0
dividend_yield           0
dy3_m_acumulado          0
dy6_m_acumulado          0
dy12_m_acumulado         0
dy3_m_media              0
dy6_m_media              0
dy12_m_media             0
dy_ano                   0
variacao_preço           0
rentab_periodo           0
rentab_acumulada         0
patrimonio_líq           0
vpa                      0
p_vpa                    0
dy_patrimonial           0
variação_patrimonial     0
rentab_patr_periodo      0
rentab_patr_acumulada    0
vacancia_fisica          0
vacancia_financeira      0
quantidade_ativos        0
dtype: int64

# 3.0 Descriptive Statistical

In [74]:
# normaliza o valor do P/VPA
df1['p_vpa'] = df1['p_vpa']/100

In [71]:
df1.describe()

,preco_atual,liquidez_diaria,dividendo,dividend_yield,dy3_m_acumulado,dy6_m_acumulado,dy12_m_acumulado,dy3_m_media,dy6_m_media,dy12_m_media,...,patrimonio_líq,vpa,p_vpa,dy_patrimonial,variação_patrimonial,rentab_patr_periodo,rentab_patr_acumulada,vacancia_fisica,vacancia_financeira,quantidade_ativos
count,301.000000,301.000000,301.000000,301.00000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,...,3.010000e+02,301.000000,301.000000,301.000000,301.000000,301.00000,301.000000,301.000000,301.000000,301.000000
mean,129.738804,19459.970100,22.057508,0.79691,2.441163,4.812193,8.135681,0.813787,0.802226,0.678007,...,3.990676e+08,2117.788505,80.940199,0.694252,-0.519169,0.16804,1.659336,7.529003,1.111993,3.275748
std,272.403341,55142.706094,126.143680,0.76406,1.461687,2.688492,5.147588,0.487246,0.448295,0.429004,...,7.524733e+08,11338.643305,62.728753,0.523093,7.879991,7.93819,14.610064,18.158892,7.267228,7.601779
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.095015e+07,-14.440000,-51.000000,0.000000,-89.990000,-89.91000,-98.890000,0.000000,0.000000,0.000000
25%,64.250000,334.000000,0.490000,0.57000,1.780000,3.570000,4.900000,0.590000,0.600000,0.410000,...,5.885315e+07,87.580000,70.000000,0.410000,-0.220000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,88.400000,2557.000000,0.740000,0.79000,2.540000,4.940000,9.010000,0.850000,0.820000,0.750000,...,1.577756e+08,99.150000,89.000000,0.690000,0.000000,0.58000,3.320000,0.000000,0.000000,0.000000
75%,100.970000,11512.000000,1.080000,1.03000,3.380000,6.770000,11.750000,1.130000,1.130000,0.980000,...,3.717563e+08,113.900000,97.000000,0.970000,0.190000,1.06000,6.660000,2.800000,0.000000,3.000000
max,2372.000000,592771.000000,783.960000,11.24000,13.620000,16.960000,18.150000,4.540000,2.830000,1.510000,...,7.710170e+09,69829.660000,1008.000000,5.080000,13.610000,14.07000,33.380000,100.000000,100.000000,72.000000


In [75]:
df1.head()

,codigo_fundo,setor,preco_atual,liquidez_diaria,dividendo,dividend_yield,dy3_m_acumulado,dy6_m_acumulado,dy12_m_acumulado,dy3_m_media,...,patrimonio_líq,vpa,p_vpa,dy_patrimonial,variação_patrimonial,rentab_patr_periodo,rentab_patr_acumulada,vacancia_fisica,vacancia_financeira,quantidade_ativos
102,ABCP11,Shoppings,73.61,830.0,0.55,0.72,2.28,4.65,9.31,0.76,...,1.119952e+09,91.77,0.80,0.60,0.02,0.62,5.14,7.50,0.0,1
286,AFHI11,Títulos e Val. Mob.,97.00,6653.0,1.15,1.19,3.90,7.93,15.36,1.30,...,2.874502e+08,95.20,1.02,1.21,-2.09,-0.91,2.26,0.00,0.0,0
284,AFOF11,Títulos e Val. Mob.,91.60,768.0,1.08,1.18,3.58,7.11,13.36,1.19,...,6.946515e+07,100.50,0.91,1.07,2.33,3.43,8.23,0.00,0.0,0
193,AIEC11,Lajes Corporativas,79.07,9534.0,0.73,0.91,2.94,6.30,11.25,0.98,...,4.891785e+07,101.38,0.78,0.72,0.04,0.76,5.83,0.00,0.0,2
3,ALMI11,Lajes Corporativas,910.00,18.0,0.00,0.00,0.00,0.00,0.00,0.00,...,2.501587e+08,2250.09,0.40,0.00,0.00,0.00,0.00,64.05,0.0,1


# 4.0 STRATEGY 

In [76]:
df1.columns

Index(['codigo_fundo', 'setor', 'preco_atual', 'liquidez_diaria', 'dividendo',
       'dividend_yield', 'dy3_m_acumulado', 'dy6_m_acumulado',
       'dy12_m_acumulado', 'dy3_m_media', 'dy6_m_media', 'dy12_m_media',
       'dy_ano', 'variacao_preço', 'rentab_periodo', 'rentab_acumulada',
       'patrimonio_líq', 'vpa', 'p_vpa', 'dy_patrimonial',
       'variação_patrimonial', 'rentab_patr_periodo', 'rentab_patr_acumulada',
       'vacancia_fisica', 'vacancia_financeira', 'quantidade_ativos'],
      dtype='object')

In [77]:
indicadores = ['codigo_fundo',
               'setor', 
               'dividendo',
               'dividend_yield',
               'p_vpa', 
               'preco_atual',
               'dy12_m_acumulado', 
               'vacancia_fisica', 
               'vacancia_financeira',                
               'quantidade_ativos']

In [78]:
df_aux = df1[indicadores]

In [97]:
def oportunidades( df ):
    aux = df
    filtros = \
    (aux['p_vpa'] < 1.0) &\
    (aux['vacancia_fisica'] == 0) &\
    (aux['dividend_yield'] > 0)
    
    return aux[filtros]
df_oportunidades = oportunidades(df_aux)

In [102]:
df_oportunidades.max()

dividendo                37.45
dividend_yield           11.24
p_vpa                     0.99
preco_atual            2200.00
dy12_m_acumulado         17.04
vacancia_fisica           0.00
vacancia_financeira       0.00
quantidade_ativos        64.00
dtype: float64

In [100]:
df1.to_csv('data/tb_fiis.csv', index=False)  
df_oportunidades.to_csv('data/tb_oportunidades.csv', index=False)

In [103]:
df_oportunidades.head()

,codigo_fundo,setor,dividendo,dividend_yield,p_vpa,preco_atual,dy12_m_acumulado,vacancia_fisica,vacancia_financeira,quantidade_ativos
284,AFOF11,Títulos e Val. Mob.,1.08,1.18,0.91,91.60,13.36,0.0,0.0,0
193,AIEC11,Lajes Corporativas,0.73,0.91,0.78,79.07,11.25,0.0,0.0,2
113,APTO11,Híbrido,0.11,1.17,0.91,9.15,0.00,0.0,0.0,0
82,ARRI11,Títulos e Val. Mob.,0.11,1.17,0.97,9.00,16.47,0.0,0.0,0
60,BARI11,Títulos e Val. Mob.,1.05,1.06,0.94,95.80,14.52,0.0,0.0,0


In [92]:
tb_fiis = Path('../data/fiis/tb_fiis.csv')  
#filepath.parent.mkdir(parents=True, exist_ok=True)  
df1.to_csv(tb_fiis)  

In [91]:
p = PureWindowsPath('C:\Users\Cliente\repos\engenhariadados\data')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-91-31966ab85ea3>, line 1)

In [79]:
media_setor = df_aux.groupby('setor').agg(['mean','std'])

In [80]:
media_setor

dividendo             dividend_yield            \
                           mean         std           mean       std   
setor                                                                  
Hospital               2.842500    3.550224       0.715000  0.157797   
Hotel                  0.483333    0.425245       0.583333  0.580029   
Híbrido              101.899516  264.669846       0.653548  0.408609   
Lajes Corporativas     2.284634    7.042124       0.539268  0.317391   
Logística              1.012069    0.934235       0.738621  0.821961   
Outros                 1.607568    2.912151       0.870270  0.436975   
Residencial            0.340000    0.325098       0.373000  0.522793   
Shoppings              1.816087    4.091535       0.537391  0.258601   
Títulos e Val. Mob.    0.880978    0.646036       1.118696  1.114948   

                        p_vpa           preco_atual              \
                         mean       std        mean         std   
setor                                                             
Hospital             0.902500  0.172892  385.805000  502.227643   
Hotel                0.963333  0.209841   99.746667   26.216083   
Híbrido              0.608871  0.426980   73.744677   67.189840   
Lajes Corporativas   0.738780  0.267434  224.414146  460.488753   
Logística            0.723103  0.376109  104.622069   34.365939   
Outros               0.801892  0.254010  154.356757  288.558777   
Residencial          0.705000  0.586804   40.246000   39.942531   
Shoppings            0.760870  0.218900  275.513913  572.363011   
Títulos e Val. Mob.  1.020652  0.973044   86.426630   98.583573   

                    dy12_m_acumulado           vacancia_fisica             \
                                mean       std            mean        std   
setor                                                                       
Hospital                    6.395000  4.349716        0.000000   0.000000   
Hotel                       7.333333  4.080445       43.300000  37.498933   
Híbrido                     6.459677  5.152261        1.719355   4.572128   
Lajes Corporativas          6.987317  4.504075       26.366341  29.492080   
Logística                   8.268966  2.401925       20.234483  27.908067   
Outros                      9.767838  4.837838        2.469459   8.223041   
Residencial                 2.864000  4.666653        3.040000   8.802424   
Shoppings                   6.215217  2.968680       10.440870  11.981319   
Títulos e Val. Mob.        10.233478  5.669972        0.000000   0.000000   

                    vacancia_financeira            quantidade_ativos  \
                                   mean        std              mean   
setor                                                                  
Hospital                       0.000000   0.000000          0.750000   
Hotel                          0.000000   0.000000          9.333333   
Híbrido                        0.908387   3.940870          4.209677   
Lajes Corporativas             6.088049  18.350044          4.097561   
Logística                      0.334483   1.259897          6.517241   
Outros                         0.515676   2.181534          6.027027   
Residencial                    0.000000   0.000000          0.600000   
Shoppings                      0.000000   0.000000          4.434783   
Títulos e Val. Mob.            0.000000   0.000000          0.065217   

                                
                           std  
setor                           
Hospital              0.500000  
Hotel                12.096832  
Híbrido               6.146136  
Lajes Corporativas   10.270844  
Logística             5.571744  
Outros               14.078128  
Residencial           1.349897  
Shoppings             5.264168  
Títulos e Val. Mob.   0.625543